In [7]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd

import findspark
findspark.init()

spark = SparkSession.builder.getOrCreate()

In [50]:
import requests as r
import zipfile as z

In [64]:
## baixar arquivo direto da fonte
zipUrl = 'http://repositorio.dados.gov.br/segrt/pensionistas/PENSIONISTAS_082021.zip'
outPath = './raw/'

req = requests.get(zipUrl)
dirOut = ( outPath + 'PENSIONISTAS.zip' )
with open(dirOut, 'wb') as f:
            f.write(req.content)
            f.close()

FileNotFoundError: [Errno 2] No such file or directory: './raw/PENSIONISTAS.zip'

### leitura dos datasets

In [ ]:

df_pensionistas = (
    spark
    .read
    .format('csv')
    .option('sep',';')
    .option('header',True)
    .load('PENSIONISTAS_082021.csv')
)

In [20]:
df_pensionistas.head()

Row(NOME DO SERVIDOR INSTITUIDOR='ABELINO ABRANCHES DE ALMEIDA', CPF SERVIDOR INSTITUIDOR='***526847**', DATA NASCIMENTO INSTITUIDOR='06041924', DATA FALECIMENTO INSTITUIDOR='29012021', MATRICULA SERVIDOR INSTITUIDOR='130000000001', NOME DO ORGAO='MINIST.DA AGRICULTURA,PECUARIA E ABAST.', SIGLA DO ORGAO='MAPA', CODIGO ORGAO SUPERIOR='0', CARGO SERVIDOR INSTITUIDOR='AGENTE ADMINISTRATIVO', NIVEL ESCOLARIDADE CARGO='NIVEL INTERMEDIARIO', CLASSE SERVIDOR INSTITUIDOR='S', PADRAO SERVIDOR INSTITUIDOR='III', REFERENCIA SERVIDOR INSTITUIDOR='00', NIVEL SERVIDOR INSTITUIDOR='000', OCORRENCIA DE INGRESSO NO SERVICO PUBLICO SERVIDOR INSTITUIDOR='NOMEACAO PARA CARGO', DATA DE OCORRENCIA DE INGRESSO NO SERVICO PUBLICO SERVIDOR INSTITUIDOR='13101945', NOME DO BENEFICIARIO='MARIA DA CRUZ DOS SANTOS', CPF DO BENEFICIARIO='***973491**', DATA DE NASCIMENTO='28011955', UF DA UPAG DE VINCULACAO='DF', TIPO DE BENEFICIARIO='CONJUGE', TIPO PENSAO='ART. 23 EC 103/2019', NATUREZA PENSAO='VITALICIA', DATA INIC

In [81]:
cols = ['NOME DO BENEFICIARIO', 
        'NOME DO ORGAO',
        'DATA DE NASCIMENTO',
        'UF DA UPAG DE VINCULACAO',
        'NATUREZA PENSAO',
        'DATA INICIO DO BENEFICIO',
        'DATA FIM DO BENEFICIO',
        'RENDIMENTO LIQUIDO',
        'PAGAMENTO SUSPENSO']


df = df_pensionistas.select(cols)

In [29]:
df.head()

Row(NOME DO BENEFICIARIO='MARIA DA CRUZ DOS SANTOS', NOME DO ORGAO='MINIST.DA AGRICULTURA,PECUARIA E ABAST.', DATA DE NASCIMENTO='28011955', UF DA UPAG DE VINCULACAO='DF', NATUREZA PENSAO='VITALICIA', DATA INICIO DO BENEFICIO='29012021', DATA FIM DO BENEFICIO='00000000', RENDIMENTO LIQUIDO='      3.039,58', PAGAMENTO SUSPENSO='NAO')

In [82]:
df = (df
 .withColumnRenamed('NOME DO BENEFICIARIO','nome')
 .withColumnRenamed('NOME DO ORGAO','orgao')
 .withColumnRenamed('DATA DE NASCIMENTO','dtnas')
 .withColumnRenamed('UF DA UPAG DE VINCULACAO','uf')
 .withColumnRenamed('NATUREZA PENSAO','natpensao')
 .withColumnRenamed('DATA INICIO DO BENEFICIO','dtiniben')
 .withColumnRenamed('DATA FIM DO BENEFICIO','dtfimben')
 .withColumnRenamed('RENDIMENTO LIQUIDO','rendLiquido')
 .withColumnRenamed('PAGAMENTO SUSPENSO','pagsuspenso')
)

In [83]:
df = df.toPandas()

In [77]:
df.to_csv('pensionistasfederal.csv', index=False)

AttributeError: 'DataFrame' object has no attribute 'to_csv'